<a href="https://colab.research.google.com/github/varshacvenkat-web/Varsha-Venkatapathy-Engineering-Portfolio-/blob/main/Article_3_Babble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import shutil
import numpy as np
import librosa
import scipy.signal
import random
import logging

# -------------------- Parameters -------------------- #
SAMPLE_RATE = 16000  # Target sample rate
FRAME_SIZE = 512     # Frame length for STFT
OVERLAP = 256        # Hop length for STFT (50% overlap)
TARGET_SHAPE = (FRAME_SIZE // 2 + 1, 100)  # (257, 100) - entire spectrogram shape before splitting
OUTPUT_DIR = "processed_data_article_3"
CLEAN_AUDIO_PATH = r"C:\Users\enhance\converted_clean_wav"  # Clean audio folder
NOISE_PARENT_DIR = r"C:\Users\enhance\Downloads\all_noises"
CHECKPOINT_DIR = "checkpoints_article_3"  # Directory for checkpoint files

# Create required directories if they don't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

# -------------------- Logging & Checkpointing -------------------- #
def setup_logger(log_file="preprocessing_article_3.log", level=logging.INFO):
    logger = logging.getLogger()
    logger.setLevel(level)
    if logger.hasHandlers():
        logger.handlers.clear()
    formatter = logging.Formatter('%(asctime)s %(levelname)s: %(message)s')
    ch = logging.StreamHandler()
    ch.setFormatter(formatter)
    logger.addHandler(ch)
    fh = logging.FileHandler(log_file)
    fh.setFormatter(formatter)
    logger.addHandler(fh)
    return logger

def checkpoint_exists(checkpoint_path):
    return os.path.exists(checkpoint_path)

def write_checkpoint(checkpoint_path, content="done"):
    with open(checkpoint_path, "w") as f:
        f.write(content)

logger = setup_logger()

# -------------------- Helper Functions -------------------- #
def clear_folder_contents(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path, exist_ok=True)
    else:
        for filename in os.listdir(folder_path):
            file_path = os.path.join(folder_path, filename)
            try:
                if os.path.isfile(file_path) or os.path.islink(file_path):
                    os.unlink(file_path)
                elif os.path.isdir(file_path):
                    shutil.rmtree(file_path)
            except Exception as e:
                logger.error(f"Failed to delete {file_path}. Reason: {e}")
    logger.info(f"Cleared contents of folder: {folder_path}")

def extract_files(parent_dir, extension=".wav"):
    files = []
    for root, _, filenames in os.walk(parent_dir):
        for file in filenames:
            if file.lower().endswith(extension):
                files.append(os.path.join(root, file))
    logger.info(f"Found {len(files)} files in {parent_dir}.")
    return files

def adjust_shape(array, target_shape):
    padded_array = np.zeros(target_shape)
    min_rows = min(array.shape[0], target_shape[0])
    min_cols = min(array.shape[1], target_shape[1])
    padded_array[:min_rows, :min_cols] = array[:min_rows, :min_cols]
    return padded_array

def mix_clean_noisy(clean_audio, noise_audio, snr_db):
    clean_power = np.mean(clean_audio ** 2)
    noise_power = np.mean(noise_audio ** 2)
    scaling_factor = np.sqrt(clean_power / (noise_power * 10 ** (snr_db / 10)))
    return clean_audio + scaling_factor * noise_audio

def compute_stft_features(audio, frame_size, overlap, sr):
    f, t, Zxx = scipy.signal.stft(audio, fs=sr, nperseg=frame_size, noverlap=overlap)
    real_part = adjust_shape(np.real(Zxx), TARGET_SHAPE)
    imag_part = adjust_shape(np.imag(Zxx), TARGET_SHAPE)
    return real_part, imag_part

def preprocess_audio(clean_path, noise_path, snr, sr=SAMPLE_RATE):
    try:
        clean_audio, _ = librosa.load(clean_path, sr=sr)
        noise_audio, _ = librosa.load(noise_path, sr=sr)

        # Adjust noise length to match clean length
        if len(noise_audio) > len(clean_audio):
            start_idx = np.random.randint(0, len(noise_audio) - len(clean_audio))
            noise_audio = noise_audio[start_idx:start_idx + len(clean_audio)]
        else:
            noise_audio = np.pad(noise_audio, (0, len(clean_audio) - len(noise_audio)), mode="wrap")

        noisy_audio = mix_clean_noisy(clean_audio, noise_audio, snr)
        clean_real, clean_imag = compute_stft_features(clean_audio, FRAME_SIZE, OVERLAP, sr)
        noisy_real, noisy_imag = compute_stft_features(noisy_audio, FRAME_SIZE, OVERLAP, sr)

        # Transpose so each row represents one time frame
        clean_real_frames = clean_real.T  # shape: (100, 257)
        clean_imag_frames = clean_imag.T  # shape: (100, 257)
        noisy_real_frames = noisy_real.T  # shape: (100, 257)
        noisy_imag_frames = noisy_imag.T  # shape: (100, 257)

        combined_clean = np.concatenate([clean_real_frames, clean_imag_frames], axis=-1)  # shape: (100, 514)
        combined_noisy = np.concatenate([noisy_real_frames, noisy_imag_frames], axis=-1)  # shape: (100, 514)
        return combined_clean, combined_noisy
    except Exception as e:
        logger.error(f"Error processing {clean_path} with {noise_path} at SNR {snr}: {e}")
        return None, None

# -------------------- Main Execution -------------------- #
# Clear output folder contents
clear_folder_contents(OUTPUT_DIR)

# Extract clean and noise files
clean_files = extract_files(CLEAN_AUDIO_PATH)
noise_files = extract_files(NOISE_PARENT_DIR)

# Determine total number of combinations (each clean x each noise x each SNR)
snr_levels = [-5, 0, 5]
total_combinations = len(clean_files) * len(noise_files) * len(snr_levels)
processed_count = 0

logger.info(f"Starting processing of {total_combinations} total combinations.")

for clean_file in clean_files:
    clean_filename = os.path.basename(clean_file).replace(".wav", "")
    clean_output_dir = os.path.join(OUTPUT_DIR, clean_filename)
    os.makedirs(clean_output_dir, exist_ok=True)

    for noise_file in noise_files:
        noise_filename = os.path.basename(noise_file).replace(".wav", "")

        for snr in snr_levels:
            checkpoint_file = os.path.join(CHECKPOINT_DIR, f"{clean_filename}_{noise_filename}_{snr}.chk")
            if checkpoint_exists(checkpoint_file):
                logger.info(f"Checkpoint exists for {clean_filename} with noise {noise_filename} at SNR {snr}. Skipping...")
                processed_count += 1
                continue

            logger.info(f"Processing {clean_filename} with {noise_filename} at SNR {snr}.")
            combined_clean, combined_noisy = preprocess_audio(clean_file, noise_file, snr)
            if combined_clean is None or combined_noisy is None:
                logger.error(f"Skipping due to error: {clean_filename} with {noise_filename} at SNR {snr}.")
                continue

            clean_combined_file = os.path.join(clean_output_dir, f"clean_combined_{snr}_{noise_filename}.npy")
            noisy_combined_file = os.path.join(clean_output_dir, f"noisy_combined_{snr}_{noise_filename}.npy")
            np.save(clean_combined_file, combined_clean)
            np.save(noisy_combined_file, combined_noisy)
            logger.info(f"Saved combined STFT features for {clean_filename} with {noise_filename} at SNR {snr} in {clean_output_dir}.")

            write_checkpoint(checkpoint_file)
            processed_count += 1
            progress_percent = (processed_count / total_combinations) * 100
            logger.info(f"Progress: {processed_count}/{total_combinations} ({progress_percent:.2f}%) complete.")

logger.info("Processing complete.")
